In [1]:
import torch

from garage import wrap_experiment
from garage.envs import GymEnv
from garage.experiment.deterministic import set_seed
from garage.sampler import LocalSampler
from garage.torch.algos import TRPO

# from garage.torch.policies import GaussianMLPPolicy

from policies.gaussian_mlp_policy import GaussianMLPPolicy

from garage.torch.value_functions import GaussianMLPValueFunction
from garage.trainer import Trainer

from TRPO_DRSOM import TRPO_DRSOM

In [2]:
@wrap_experiment
def trpo_pendulum(ctxt=None, seed=1):
    set_seed(seed)
    env = GymEnv('Pendulum-v0')
    trainer = Trainer(ctxt)
    policy = GaussianMLPPolicy(env.spec,
                               hidden_sizes=[32, 32],
                               hidden_nonlinearity=torch.tanh,
                               output_nonlinearity=None)


    value_function = GaussianMLPValueFunction(env_spec=env.spec,
                                              hidden_sizes=(32, 32),
                                              hidden_nonlinearity=torch.tanh,
                                              output_nonlinearity=None)
    sampler = LocalSampler(agents=policy,
                           envs=env,
                           max_episode_length=env.spec.max_episode_length)
    algo = TRPO_DRSOM(env_spec=env.spec,
                      policy=policy,
                      value_function=value_function,
                      sampler=sampler,
                      discount=0.99,
                      center_adv=False)
    trainer.setup(algo, env)
    trainer.train(n_epochs=10, batch_size=1024)

In [3]:
trpo_pendulum(seed=1)

2022-08-16 16:54:27 | [trpo_pendulum] Logging to d:\Github\DRSOM-for-RL\data/local/experiment/trpo_pendulum_3


d:\Anaconda\lib\site-packages\garage\experiment\deterministic.py:36: UserWarning: Enabeling deterministic mode in PyTorch can have a performance impact when using GPU.
  warnings.warn(


2022-08-16 16:54:27 | [trpo_pendulum] Obtaining samples...
tensor([0.0007, 0.0007])
2022-08-16 16:54:28 | [trpo_pendulum] epoch #0 | Saving snapshot...
2022-08-16 16:54:28 | [trpo_pendulum] epoch #0 | Saved
2022-08-16 16:54:28 | [trpo_pendulum] epoch #0 | Time 0.40 s
2022-08-16 16:54:28 | [trpo_pendulum] epoch #0 | EpochTime 0.40 s
----------------------------------  ----------
Evaluation/AverageDiscountedReturn   -664.165
Evaluation/AverageReturn            -1612.72
Evaluation/Iteration                    0
Evaluation/MaxReturn                -1496.35
Evaluation/MinReturn                -1688.5
Evaluation/NumEpisodes                  6
Evaluation/StdReturn                   73.6627
Evaluation/TerminationRate              0
TotalEnvSteps                        1200
----------------------------------  ----------
tensor([0.0232, 0.0232])
2022-08-16 16:54:28 | [trpo_pendulum] epoch #1 | Saving snapshot...
2022-08-16 16:54:28 | [trpo_pendulum] epoch #1 | Saved
2022-08-16 16:54:28 | [trpo_p